In [34]:
import os, sys
CURRENT_DIR = os.getcwd()
FORECASTING_MODULE_DIR = os.path.abspath(os.path.join(CURRENT_DIR, '..'))
sys.path.append(FORECASTING_MODULE_DIR)
print(f"Adding {FORECASTING_MODULE_DIR} to sys.path")
import pandas as pd
from sklearn.metrics import mean_absolute_error, mean_squared_error, root_mean_squared_error
from plantbrain_fastml.models.forecasters.arima import ArimaForecaster
from plantbrain_fastml.models.forecasters.exp_smotthing import ExpSmoothingForecaster

Adding /Users/aditya/Desktop/fastml/plantbrain-fastml to sys.path


In [35]:
import warnings
warnings.filterwarnings("ignore")

In [36]:
df = pd.read_csv("/Users/aditya/Desktop/fastml/plantbrain-fastml/tests/forecast_test.csv")          # or any other source
df["Date"] = pd.to_datetime(df["Date"])
df = (
    df[["Date", "Close"]]
    .dropna(subset=["Close"])
    .sort_values("Date")
    .set_index("Date")
)

In [37]:
split = int(len(df) * 0.8)
x_train = df.index[:split]
x_test  = df.index[split:]
y_train = df["Close"].iloc[:split]
y_test  = df["Close"].iloc[split:]

In [43]:
model = ExpSmoothingForecaster(
    forecast_horizon=len(y_test),   # predict the whole hold-out at once
    transformation="power_transformer",   # or None
    arima_orders=None,              # let auto_arima decide
)

In [44]:
model.train(X=x_train, y=y_train)

In [45]:
preds = model.predict(steps=len(y_test))

In [46]:
model.evaluate(X=None, y=y_test)

{'rmse': np.float64(586.2535711512827),
 'mape': nan,
 'mae': np.float64(464.4526415718593),
 'mse': nan}

In [33]:
# works
# model.save("/Users/aditya/Desktop/fastml/plantbrain-fastml/tests/arima_model.pkl")

In [47]:
model.hypertune(
    X=x_train,
    y=y_train)

[I 2025-06-23 16:59:45,019] A new study created in memory with name: no-name-d7ab283a-0889-4723-96f4-70a70ab342c2
[I 2025-06-23 16:59:45,331] Trial 0 finished with value: 74.59127625166207 and parameters: {'trend': None, 'damped_trend': False, 'seasonal': 'mul', 'seasonal_periods': 4}. Best is trial 0 with value: 74.59127625166207.
[I 2025-06-23 16:59:45,631] Trial 1 finished with value: 74.59127625166207 and parameters: {'trend': 'add', 'damped_trend': False, 'seasonal': 'mul', 'seasonal_periods': 16}. Best is trial 0 with value: 74.59127625166207.
[I 2025-06-23 16:59:45,928] Trial 2 finished with value: 74.59127625166207 and parameters: {'trend': 'mul', 'damped_trend': True, 'seasonal': None}. Best is trial 0 with value: 74.59127625166207.
[I 2025-06-23 16:59:46,229] Trial 3 finished with value: 74.59127625166207 and parameters: {'trend': None, 'damped_trend': False, 'seasonal': 'add', 'seasonal_periods': 20}. Best is trial 0 with value: 74.59127625166207.
[I 2025-06-23 16:59:46,526]

{'trend': None,
 'damped_trend': False,
 'seasonal': 'mul',
 'seasonal_periods': 4}